In [27]:
# Imports
import logging
import pandas as pd
# import requests
import sqlite3
import sklearn
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from scipy.stats import pearsonr




#logging info
logging.basicConfig(level = logging.DEBUG)



dbName = "../rest_server/medisch_centrum_randstad/db.sqlite3"
tableName = "rest_api_netlify"
url = "http://localhost:8080/medish_centrum_randstad/api/netlify?page=1"
csvFile = "rest_server_new/medisch_centrum_randstad/data/data.csv"

##########################
### Read from REST API ###
##########################
# logging.info("Extract from REST API")
# response = requests.get(url)
# file_contents= response.json()  #dictionary
# logging.info("Load API Response into Pandas dataframe")
# df = pd.DataFrame.from_dict(file_contents['data']) #all the needed info was condensed into one data column called 'data'
# df3 = df.copy() #keep original for df3 
# logging.debug(df.head())
# logging.info("Load dataset in Database")

######################
### Read from .csv ###
######################
#df pd.read_csv('csvFile',skipinitialspace=True)
#df3 = df.copy() #keep original for df3 
#logging.info("Load dataset in Database")

#


In [41]:
###########################
### Read from SQlite3 db created in our pipeline ###
############################
dbConnection = sqlite3.connect(dbName)

#query db and write to pd:
dfFromDB = pd.read_sql_query(f"SELECT * FROM {'df2'}", dbConnection)
#sql adds index, remove:
df2 = dfFromDB.drop('index', axis=1)
pd.set_option('display.max_columns', 10)
display(df2.head(2))
# display(df2.info)

df=df2.copy()
y_axis_test = 'lifespan'



dbConnection.close()

,genetic,length,mass,exercise,smoking,alcohol,lifespan,sugar,BMI
0,73.9,185.0,99.7,0.9,0.0,2.4,73.1,6.9,29.1
1,86.0,172.0,105.4,1.8,8.1,0.4,85.0,4.2,35.6


In [45]:
X = 'genetic'
df_x=df[[y_axis_test,X]]
df_x=df
#display(df_x.info())
display(df_x.head())

,lifespan,genetic
0,73.1,73.9
1,85.0,86.0
2,81.6,83.3
3,81.0,82.8
4,75.0,78.7


In [48]:
df_cut = df2.copy()

## the IQR clipping for outliers 
# Computing IQR
Q1 = df_cut.quantile(0.25)
Q3 = df_cut.quantile(0.75)
IQR = Q3 - Q1

# Clipping the IQR*|15.*IQD|
df_cut = df_cut[~((df_cut < (Q1 - 1.5 * IQR)) |(df_cut > (Q3 + 1.5 * IQR))).any(axis=1)]

In [49]:
df_x = df2[['lifespan','mass']]

df5 = df_x [['lifespan','mass']]
df5 = df5.join(df_cut[['genetic','exercise','smoking','alcohol','BMI']])

display(df5.head())

,lifespan,mass,genetic,exercise,smoking,alcohol,BMI
0,73.1,99.7,73.9,0.9,0.0,2.4,29.1
1,85.0,105.4,86.0,1.8,8.1,0.4,35.6
2,81.6,111.4,83.3,1.1,0.8,4.6,36.0
3,81.0,111.4,82.8,4.7,11.8,1.0,41.4
4,75.0,71.6,78.7,1.5,8.3,4.9,22.6
